In [19]:
# this notebook will give you the appotunity to work with dynamic data
# if takes final.csv file with data which is not separated by dates 
# and give you back date_data.csv file - date separated dataset

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import trimboth
import ast
import time
import datetime
import geopy
import matplotlib.pyplot as plt
from tqdm import tqdm
import geopy.distance
from tqdm._tqdm_notebook import tqdm_notebook
tqdm.pandas(desc="Bar")

In [3]:
import plotly.offline as pyo
import chart_studio.plotly as chspy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px

init_notebook_mode(connected=True)

In [4]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

In [5]:
flats = pd.read_csv('final.csv', index_col=0)

In [6]:
spb = flats[(flats.trade_type == 'sale')&\
            ((flats.city == 'Санкт-Петербург')| (flats.city == 'Ленинградская область'))]

In [7]:
spb['date'] = spb.date_of_adding_to_db

In [8]:
#made list of unique dates
date_list = spb.date.unique()

In [9]:
date_df = []

In [10]:
for date in tqdm(date_list):
    younger_ad = spb[spb.date<=date]
    most_actual_ad = younger_ad[younger_ad.groupby('cian_id')['date'].transform(max) == younger_ad['date']]
    max_active_ads = most_actual_ad[most_actual_ad.active == True]   
    date_df.append(max_active_ads)

100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [00:35<00:00,  1.85it/s]


In [11]:
res = pd.concat(date_df, keys=date_list, axis=1)

In [12]:
res.keys().get_level_values(0)

Index(['2020-02-27', '2020-02-27', '2020-02-27', '2020-02-27', '2020-02-27',
       '2020-02-27', '2020-02-27', '2020-02-27', '2020-02-27', '2020-02-27',
       ...
       '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08',
       '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08', '2020-05-08'],
      dtype='object', length=3564)

In [13]:
mean_square_price = []
square_price = []
for date in res.keys().get_level_values(0).unique():
    mean_square_price.append(res[date].price_per_sq_meter.mean())
    square_price.append(res[date].price_per_sq_meter)

In [14]:
px.line(x = res.keys().get_level_values(0).unique(), y=mean_square_price)

In [15]:
res.to_csv('date_data.csv')

In [16]:
lst = []
for date in res.keys().get_level_values(0).unique():
    lst.append(res[date]['cian_id'].nunique())
#     square_price.append(res[date].price_per_sq_meter)